In [3]:
#Load modules

import networkx as nx
# import matplotlib.pyplot as plt
# %matplotlib inline

import pandas as pd

from numpy.random import choice


In [4]:

def select_nodes_by_kw(graph, keyword_list = []):
    """
    This function returns a list of all nodes whose name contains some of the keywords.
    
    graph: networkx graph object
    
    """
    result_list = []
    for kw in keyword_list:
        result_list += [label for label in graph.nodes() if kw in label]
    return result_list

def second_neighbors(graph, nodes, second_order = True):
    """
    graph: networkx graph
    nodes: list of node names
    second_order: If 'False' search stops at direct (1st order) neighborhood. 
    Returns subgraph with up to second neighbors of node, and edges between them.                                                                              
    """
    neighbor_list = [nodes]

    for i in range(len(nodes)):
        neighbor_list += [graph.neighbors(nodes[i])]
        
        if second_order == True:
            for n in graph.neighbors(nodes[i]):
                    neighbor_list += [graph.neighbors(n)]


    return list(set([item for sublist in neighbor_list for item in sublist]))

def wrap_by_word(s, n):
    '''returns a string where \\n is inserted between every n words'''
    a = s.split()
    ret = ''
    for i in range(0, len(a), n):
        ret += ' '.join(a[i:i+n]) + ' \n '

    return ret


In [3]:
# Columns of the datasets
# Geo_cat = ['CHI ID COM']#'BRA ID MIR'
# HS_cat = ['HS ID 6']
# ind_cat = ['CIIU ID 4']

Geo_cat = ['BRA ID MIR'] #'BRA ID MIR'
HS_cat = ['HS ID 4']
ind_cat = ['CNAE ID C', 'CBO ID F']


var = 'Total Yearly Wages' # 'output_USD'
var_str = '_w'
var_lab_str = 'Wages in '# 'Output of ' #'Wages in '

In [1]:
#Choose a configuration for further aggregation.

# ctry_str = Geo_cat[0][:3]
# geo_str = Geo_cat[0][-3:]

# confs = [('4', '2', '2'),
#          ('4', '4', '2')]

# confs = [('0', 'G', '4', '4'),
#          ('GG', 'D', '4', '4'),
#          ('SGP', 'S', '4', '4'),
#          ('SGP', 'S', '4', '4')]

# for conf in confs:
# #     (CIIU_lev, HS_Exp, HS_Imp) = conf
#     (CBO_lev, CNAE_lev, HS_Exp, HS_Imp) = conf

#     col_name = ctry_str+'_'+geo_str+'_'+'_'.join([c for c in conf])+var_str
#     print col_name

In [8]:
# Path where structure reference tables are:
path = 'data/disagg_struct_refs/formatted/'

GEO_chi = pd.read_csv('data/disagg_struct_refs/formatted/GEO_chi.csv')
GEO_bra = pd.read_csv('data/disagg_struct_refs/formatted/GEO_bra.csv')

GEO_bra['BRA label EST'] = GEO_bra['BRA ID EST']
GEO_bra['BRA label MER'] = GEO_bra['BRA ID MER']
GEO_bra['BRA label MIR'] = GEO_bra['BRA ID MIR']

In [36]:
import itertools
from collections import OrderedDict
from numpy import array

geo_levs = ['CHI_REG', 'CHI_PRO', 'CHI_COM']
geo_levs = ['CHI_COM']
J = len(geo_levs)

# This dictionary summarises the possible configurations for the experiment.
# 'name' points to the variable names, eg. ['Wages', 'Exports', 'Imports']
# 'dataset' tells us the name of the dataset that each of the variables above belong to.
# 'levels' tells the disaggregation levels we would like each variable to be analised in.
# 'agg_str' is the name of the dataset used for reference of the disaggregation, for each of the variables
var_dict = {'name': ['Wages', 'Exports', 'Imports'], 
       'dataset': ['prd', 'trd', 'trd'], 
       'levels': [['CI1', 'CI2','CI4'], ['HS1', 'HS2','HS4'], ['HS1', 'HS2']],
           'agg_str' : ['CIIU', 'HS', 'HS']}

var_dict = {'name': ['Wages', 'Output', 'Exports', 'Imports'], 
       'dataset': ['prd','prd', 'trd', 'trd'], 
       'levels': [['CI1', 'CI2', 'CI4'],['CI1', 'CI2', 'CI4'], ['HS1', 'HS2', 'HS4'], ['HS1', 'HS2', 'HS4']],
#            'agg_ref' : [CIIU, CIIU, HS, HS],
           'agg_str' : ['CIIU','CIIU', 'HS', 'HS']}

# The brazil case may be more special cause aggregation of the production dataset is over two directions
# var_dict = {'name': ['Wages', 'Output', 'Exports', 'Imports'], 
#        'dataset': ['prd','prd', 'trd', 'trd'], 
#        'levels': [['CN1', 'CN2', 'CN4'],['CN1', 'CN2', 'CN4'], ['HS1', 'HS2', 'HS4'], ['HS1', 'HS2', 'HS4']],
# #            'agg_ref' : [CIIU, CIIU, HS, HS],
#            'agg_str' : ['CNAE','CNAE', 'HS', 'HS']}


# all possible combination of levels. They could be filtered afterwards...
confs = list(itertools.product(*var_dict['levels']))

#select(sample) every three
confs = array(confs)[[i for i in range(len(confs)) if i%6 == 0]]
K = len(confs)


In [145]:
import seaborn as sns
import math

from bokeh.io import show, output_file, save
from bokeh.plotting import figure
from bokeh.models import GraphRenderer, StaticLayoutProvider, Circle, MultiLine, NodesOnly, NodesAndLinkedEdges, EdgesAndLinkedNodes
from bokeh.models.graphs import from_networkx
from bokeh.models.tools import HoverTool

from numpy import power

# Maximum degree that a network can have.
max_d = 10

# String that identifies the right 'weight' column in the edges dataset. 
# also used as prefix in the configuration name
geo = 'CHI_COM'


# Column of name and id of geographical units.
places_id = 'CHI ID COM'#'BRA ID MIR' #'CHI ID COM'
places_label = 'CHI label COM'#'BRA label MIR' #'CHI label COM'
GEO_ref = GEO_chi #GEO_bra #GEO_chi

ctry_str = 'CHI'

path = 'data/new/'
path = './../../../../../media/miglesia/Elements/corr_st_data/'


l = len(geo_levs)

k = .1

for j in range(len(confs)):

#     try:
        
    conf = confs[j]


#     (CIIU_lev, HS_Exp, HS_Imp) = conf
#     (CBO_lev, CNAE_lev, HS_Exp, HS_Imp) = conf
    print conf

#     col_name = ctry_str+'_'+geo_str+'_'+'_'.join([c for c in conf])+var_str
    conf_name = geo+'_'+'_'.join([var_dict['name'][i][0]+conf[i][-1] for i in range(len(var_dict['name']))])


    nodes = pd.read_csv(path+conf_name+'_nodes.csv', index_col = 0)
    edges = pd.read_csv(path+conf_name+'_edges.csv', index_col = 0)

        ### Max degree = max_d. Parameter used to trim the network.
    df1 = edges.groupby('label_x').apply(lambda x: x.nlargest(max_d, 'weight_'+geo)).reset_index(drop = True)
    df2 = edges.groupby('label_y').apply(lambda x: x.nlargest(max_d, 'weight_'+geo)).reset_index(drop = True)
    edges = df1.merge(df2, on = ['label_x', 'label_y', 'weight_'+geo]).reset_index(drop = True)

#     Add info of districts with most volume (CHI)
    ###
    concat_data = pd.read_csv(path+conf_name+'_cc_data.csv', index_col = 0).reset_index()
    df = concat_data.sort_values(by = ['node_name', 'value_USD'])
    
#     node_grouped = df.groupby('node_name')['value_USD']
    node_totals = df.groupby('node_name')['value_USD'].sum().reset_index().rename({'value_USD': 'node_value_USD'}, axis = 1)
    
    df = df.merge(node_totals)

    df['rk'] = df.groupby('node_name')['value_USD'].rank(method = 'first', ascending = False)
    df['pct'] = df['value_USD'].div(df['node_value_USD']).multiply(100).dropna().round(1)

    df = df.loc[df.rk <= 3].merge(GEO_ref[[places_id, places_label]].drop_duplicates())
    df['info_'] = df[places_label] + ' ('+df.pct.astype(str)+'%)'
    df = df.sort_values(by = 'node_name')
    df = pd.DataFrame(df.set_index(['node_name', 'rk'])[['info_']].unstack())
    # df
    df.columns = df.columns.droplevel(0)

    df['info'] = df[1.0] +', '+ df[2.0]+', '+df[3.0]
    df = df.reset_index()[['node_name', 'info']]

#     nodes = nodes.merge(df[['node_name', 'info']])
    nodes = nodes.merge(df.set_index('node_name')[['info']], left_index = True, right_index = True)

#     ###

    #color
    from matplotlib import colors
    nodes['color_code'] = nodes.att_A.rank(method = 'dense')
    w = plt.cm.get_cmap('rainbow', len(nodes.color_code.unique()))
    
    colors_hex = []
    for i in range(w.N):
        rgb = w(i)[:3] # will return rgba, we take only first 3 so we get rgb
        colors_hex += [colors.rgb2hex(rgb)]

    colors_hex = [w.encode('UTF8') for w in colors_hex]
    code_hex = pd.DataFrame([nodes.color_code.unique(), colors_hex], index = ['color_code', 'color']).T

    #size

    value = nodes['node_value_USD']
    # nodes['node_size'] = np.sqrt((value/value.median()).values)
    nodes['node_size'] = 10 * power((value/value.median()).values, 1/2.)

#     nodes = nodes.merge(code_hex)
    nodes = nodes.reset_index().merge(code_hex, on = 'color_code').set_index('node_name')
    
    ###

    G = nx.from_pandas_dataframe(edges, 'label_x', 'label_y', 'weight_'+geo)



    ### Subgraph - neighborhood
    # Random choice of some nodes...
    nodes_choice = choice(G.nodes(), 5)

    for node in nodes_choice:

        ix = nodes.loc[node].index.values[0]

        G_s = G.subgraph(second_neighbors(G, [node], second_order=True))  
        nodes_s = nodes.loc[G_s.nodes()].reset_index()
        edges_s = edges.loc[(edges.label_x.isin(G_s.nodes())) & (edges.label_y.isin(G_s.nodes()))]


        # nx.write_gexf(G,'networks/'+col_name+'.gexf')

    #         print 'nodes: '+str(len(G.nodes()))
    #         print 'edges: '+str(len(G.edges()))

        # Subset of nodes
#         nodes_s = nodes.set_index('node_name').loc[G_s.nodes()].reset_index()

        #Perform Graph Drawing
    #     pos = nx.spring_layout(G, k=k)
    #     nx.draw(G, pos, ax = ax, node_size = 5*nodes.node_size.values, alpha=.6, node_color= list(nodes.color.values), edgelist = [])

        #     Bokeh drawing
        ####

        N = len(nodes_s)
        node_indices = list(range(N))
        nodes_s['node_name_'] = [wrap_by_word(s, 5) for s in nodes_s['node_name'].values]

        plot = figure(title=col_name, x_range=(-1.1,1.1), y_range=(-1.1,1.1))

        #####

        graph = from_networkx(G_s, nx.spring_layout, center=(0,0), k = k)

        graph.node_renderer.data_source.add(list(nodes_s['color'].values), 'color')
        graph.node_renderer.data_source.add(list(nodes_s['node_name_'].values), 'node_name_')
        graph.node_renderer.data_source.add(list(nodes_s['node_value_USD'].values), 'node_value_USD')
        graph.node_renderer.data_source.add(list(nodes_s['info'].values), 'info')
        graph.node_renderer.data_source.add(list(.5*nodes_s['node_size'].values), 'node_size')

        graph.node_renderer.glyph = Circle(size='node_size', fill_color='color')
        graph.node_renderer.selection_glyph = Circle(size=15, fill_color='#808000')
        graph.node_renderer.hover_glyph = Circle(size=15, fill_color='#808000')

        graph.edge_renderer.glyph = MultiLine(line_color="#CCCCCC", line_alpha=0.5, line_width=1)
        graph.edge_renderer.selection_glyph = MultiLine(line_color='#808000', line_width=5)
        graph.edge_renderer.hover_glyph = MultiLine(line_color='#808000', line_width=5)
        graph.selection_policy = NodesAndLinkedEdges()
        graph.selection_policy = EdgesAndLinkedNodes()

        plot.renderers.append(graph)

        hover = HoverTool(tooltips=dict(main_place="@info", value="@node_value_USD", activity="@node_name_"))
        # hover = HoverTool(tooltips = [('Info', '@node_name'),('(x,y)', '($x, $y)')])

        plot.add_tools(hover)

        output_file('./Figures/6_neighbhoods/'+col_name+'_'+str(ix).zfill(3)+'.html')
        save(plot)
#     except IOError:
#         print 'IOError at '+str('data/node_refs/'+conf_name+'_nodes.csv')

['CI1' 'CI1' 'HS1' 'HS1']
Index([u'Exports of 'Live Animals; Animal Products'',
       u'Exports of 'Vegetable Products'',
       u'Exports of 'Animal Or Vegetable Fats And Oils And Their Cleavage Products; Prepared Edible Fats; Animal Or Vegetable Waxes''],
      dtype='object', name=u'node_name')
Index([u'Exports of 'Live Animals; Animal Products'',
       u'Imports of 'Live Animals; Animal Products'',
       u'Exports of 'Vegetable Products''],
      dtype='object', name=u'node_name')
['CI1' 'CI1' 'HS4' 'HS1']
Index([u'Exports of '0101 - Horses, asses, mules and hinnies; live'',
       u'Exports of '0102 - Bovine animals; live'',
       u'Exports of '0103 - Swine; live''],
      dtype='object', name=u'node_name')
Index([u'Exports of '0101 - Horses, asses, mules and hinnies; live'',
       u'Exports of '0102 - Bovine animals; live'',
       u'Exports of '0103 - Swine; live''],
      dtype='object', name=u'node_name')
['CI1' 'CI2' 'HS2' 'HS1']
Index([u'Exports of '01 - Animals; live''

In [146]:
# code_hex['color_code'] = code_hex['color_code'].astype(float)


# code_hex.dtypes

In [132]:
edges#.groupby('label_x').apply(lambda x: x.nlargest(max_d, 'weight_'+geo_str)).reset_index(drop = True)

,index_x,label_x,label_y,weight_CHI_COM,index_y
0,3,Exports of 'Base Metals And Articles Of Base M...,Exports of 'Mineral Products',1.874387,3
1,6,Exports of 'Base Metals And Articles Of Base M...,Exports of 'Products Of The Chemical Or Allied...,1.718824,6
2,4,Exports of 'Base Metals And Articles Of Base M...,"Exports of 'Natural Or Cultured Pearls, Precio...",1.242880,4
3,1,Exports of 'Base Metals And Articles Of Base M...,Exports of 'Live Animals; Animal Products',0.893584,1
4,31,Exports of 'Base Metals And Articles Of Base M...,Output of 'CIIU seccion 2',0.886588,31
5,18,Exports of 'Base Metals And Articles Of Base M...,Imports of 'Products Of The Chemical Or Allied...,0.818204,18
6,48,Exports of 'Base Metals And Articles Of Base M...,Wages of 'CIIU seccion 2',0.792296,48
7,34,Exports of 'Base Metals And Articles Of Base M...,Output of 'CIIU seccion 5',0.588680,34
8,62,Exports of 'Live Animals; Animal Products',Exports of 'Wood And Articles Of Wood; Wood Ch...,1.834539,62
9,75,Exports of 'Live Animals; Animal Products',Output of 'CIIU seccion 1',1.632827,75


In [33]:
pd.read_csv('data/new/'+conf_name+'_log_corr.csv', index_col = 0)


,Exports of 'Animal Or Vegetable Fats And Oils And Their Cleavage Products; Prepared Edible Fats; Animal Or Vegetable Waxes',Exports of 'Arms And Ammunition; Parts And Accessories Thereof',"Exports of 'Articles Of Stone, Plaster, Cement, Asbestos, Mica Or Similar Materials; Ceramic Products; Glass And Glassware'",Exports of 'Base Metals And Articles Of Base Metal',"Exports of 'Footwear, Headgear, Umbrellas, Sun Umbrellas, Walking Sticks, Seat-Sticks, Whips, Riding-Crops And Parts Thereof; Prepared Feathers And Articles Made Therewith; Artificial Flowers; Articles Of Human Hair'",Exports of 'Live Animals; Animal Products',"Exports of 'Machinery And Mechanical Appliances; Electrical Equipment; Parts Thereof; Sound Recorders And Reproducers, Television Image And Sound Recorders And Reproducers, And Parts And Accessories Of Such Articles'",Exports of 'Mineral Products',Exports of 'Miscellaneous Manufactured Articles',"Exports of 'Natural Or Cultured Pearls, Precious Or Semi-Precious Stones, Precious Metals, Metals Clad With Precious Metal, And Articles Thereof; Imitation Jewellery; Coin'",...,Wages of 'CIIU seccion 2',Wages of 'CIIU seccion 20',Wages of 'CIIU seccion 21',Wages of 'CIIU seccion 3',Wages of 'CIIU seccion 4',Wages of 'CIIU seccion 5',Wages of 'CIIU seccion 6',Wages of 'CIIU seccion 7',Wages of 'CIIU seccion 8',Wages of 'CIIU seccion 9'
Exports of 'Animal Or Vegetable Fats And Oils And Their Cleavage Products; Prepared Edible Fats; Animal Or Vegetable Waxes',1.000000,0.104838,0.360671,0.550118,0.454870,0.398582,0.449944,0.250478,0.358156,0.281717,...,0.361415,0.377973,0.356264,0.389450,0.308749,0.316131,0.474191,0.393349,0.376562,0.321524
Exports of 'Arms And Ammunition; Parts And Accessories Thereof',0.104838,1.000000,0.477156,0.532411,0.087664,0.344543,0.544932,0.465394,0.485415,0.428396,...,0.161946,0.299954,NaN,0.592514,0.515218,0.562939,0.438819,0.577688,0.508197,0.610108
"Exports of 'Articles Of Stone, Plaster, Cement, Asbestos, Mica Or Similar Materials; Ceramic Products; Glass And Glassware'",0.360671,0.477156,1.000000,0.270373,0.543714,0.170166,0.541754,0.168631,0.581304,0.167939,...,0.208649,0.277389,0.256025,0.428806,0.204093,0.200799,0.344759,0.400054,0.293509,0.325074
Exports of 'Base Metals And Articles Of Base Metal',0.550118,0.532411,0.270373,1.000000,0.394557,0.489346,0.634768,0.567483,0.395002,0.398770,...,0.471351,0.388158,0.194279,0.560034,0.553496,0.520243,0.690180,0.639515,0.651703,0.563491
"Exports of 'Footwear, Headgear, Umbrellas, Sun Umbrellas, Walking Sticks, Seat-Sticks, Whips, Riding-Crops And Parts Thereof; Prepared Feathers And Articles Made Therewith; Artificial Flowers; Articles Of Human Hair'",0.454870,0.087664,0.543714,0.394557,1.000000,-0.016323,0.438861,0.295510,0.419076,0.282724,...,0.305842,0.481249,0.853398,0.472689,0.449319,0.287933,0.436505,0.534819,0.377439,0.459907
Exports of 'Live Animals; Animal Products',0.398582,0.344543,0.170166,0.489346,-0.016323,1.000000,0.296337,0.416532,0.376229,0.292016,...,0.214957,0.255248,0.238892,0.383755,0.404926,0.398045,0.426667,0.448899,0.491675,0.394802
"Exports of 'Machinery And Mechanical Appliances; Electrical Equipment; Parts Thereof; Sound Recorders And Reproducers, Television Image And Sound Recorders And Reproducers, And Parts And Accessories Of Such Articles'",0.449944,0.544932,0.541754,0.634768,0.438861,0.296337,1.000000,0.240884,0.545081,0.058917,...,0.373490,0.460099,0.254956,0.782668,0.451544,0.478351,0.731083,0.750161,0.737261,0.705702
Exports of 'Mineral Products',0.250478,0.465394,0.168631,0.567483,0.295510,0.416532,0.240884,1.000000,0.275148,0.603489,...,0.506125,0.500048,0.447030,0.242454,0.470832,0.266306,0.428600,0.376066,0.338520,0.343260
Exports of 'Miscellaneous Manufactured Articles',0.358156,0.485415,0.581304,0.395002,0.419076,0.376229,0.545081,0.275148,1.000000,0.045058,...,0.219772,0.377553,-0.063311,0.499426,0.379053,0.366291,0.455254,0.499745,0.443512,0.370980
"Exports of 'Natural Or Cultured Pearls, Precious Or

### Trying to add some info to the nodes...

In [ ]:


k = .05
max_d = 10

for conf in confs:
    (CIIU_lev, HS_Exp, HS_Imp) = conf
    col_name = ctry_str+'_'+geo_str+'_'+'_'.join([c for c in conf])+var_str
    print col_name
    
    for i in range(10):


        nodes = pd.read_csv('data/node_refs/'+col_name+'_nodes_meta.csv', index_col = 0)
        print len(nodes)
        edges = pd.read_csv('data/node_refs/'+col_name+'_edges.csv', index_col = 0)

        ### Max degree = max_d
        df1 = edges.groupby('label_x').apply(lambda x: x.nlargest(max_d, 'weight_COM')).reset_index(drop = True)
        df2 = edges.groupby('label_y').apply(lambda x: x.nlargest(max_d, 'weight_COM')).reset_index(drop = True)
        edges = df1.merge(df2, on = ['label_x', 'label_y', 'weight_COM']).reset_index(drop = True)

        G = nx.from_pandas_dataframe(edges, 'label_x', 'label_y', 'weight_'+geo_str)


        ### Subgraph - neighborhood
    #         nodes_sel = select_nodes_by_kw(G, ['Wine','grape', 'Grape','bebidas fermentadas'])
    #     nodes_sel = [ 'Output ofPesca maritima ']
    #     nodes_sel = select_nodes_by_kw(G, ['Silvicultura'])
        nodes_sel = choice(G.nodes(), 1)

        G = G.subgraph(second_neighbors(G, nodes_sel, second_order=True))  
        nodes = nodes.set_index('node_name').loc[G.nodes()].reset_index()
        edges = edges.loc[(edges.label_x.isin(G.nodes())) & (edges.label_y.isin(G.nodes()))]

        # nx.write_gexf(G,'networks/'+col_name+'.gexf')

    #     Networkx drawing
        #Perform Graph Drawing
        plt.figure(figsize=(8, 8))
        pos = nx.spring_layout(G, k=0.15)

        nx.draw(G, pos, node_size = 10 * nodes.node_size.values, alpha=.6, node_color= list(nodes.color.values), edgelist = [])
        nx.draw_networkx_edges(G, pos, edge_color='.7', alpha=.3)
        #     plt.savefig('../DataViva/Figures/'+'_'+kw+'_'+name)
        plt.show()

        ####

    #     Bokeh drawing
        ####

        N = len(nodes)
        node_indices = list(range(N))
        # sizes = np.linspace(10, 20, N)
        # node_sizes = np.sqrt(nodes['node_size'].values)

        plot = figure(title="Networkx Integration Demonstration", x_range=(-1.1,1.1), y_range=(-1.1,1.1))

        #####

        graph = from_networkx(G, nx.spring_layout, center=(0,0), k = k)

        graph.node_renderer.data_source.add(list(nodes['color'].values), 'color')
        graph.node_renderer.data_source.add(list(nodes['node_name'].values), 'node_name')
        graph.node_renderer.data_source.add(list(2*nodes['node_size'].values), 'node_size')

        graph.node_renderer.glyph = Circle(size='node_size', fill_color='color')
        graph.node_renderer.selection_glyph = Circle(size=15, fill_color='#808000')
        graph.node_renderer.hover_glyph = Circle(size=15, fill_color='#808000')

        graph.edge_renderer.glyph = MultiLine(line_color="#CCCCCC", line_alpha=0.2, line_width=1)
        graph.edge_renderer.selection_glyph = MultiLine(line_color='#808000', line_width=5)
        graph.edge_renderer.hover_glyph = MultiLine(line_color='#808000', line_width=5)
        graph.selection_policy = NodesAndLinkedEdges()

        plot.renderers.append(graph)

        hover = HoverTool(plot=plot, tooltips=dict(act="@node_name", value="@node_size"))
        # hover = HoverTool(tooltips = [('Info', '@node_name'),('(x,y)', '($x, $y)')])

        plot.add_tools(hover)

#         output_file("networkx_graph.html")
        # output_notebook()
        show(plot)

        nodes = nodes.sort_values(by = 'node_size', ascending = False)
        sns.barplot(x='node_size',y='node_name',palette=nodes['color'], data=nodes, edgecolor = 'None')


In [ ]:
conf